In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv("books_train.csv")
df.head(5)


,title,description,categories
0,ساختار نظریه‌های جامعه‌شناسی ایران,«ساختار نظریه‌های جامعه‌شناسی ایران» نوشته ابو...,جامعه‌شناسی
1,جامعه و فرهنگ کانادا,«جامعه و فرهنگ کانادا» از مجموعه کتاب‌های «جام...,جامعه‌شناسی
2,پرسش از موعود,پرسش‌های مختلفی درباره زندگی و شخصیت امام مهدی...,کلیات اسلام
3,موج، دریا,«موج دریا» به قلم مهری ماهوتی (-۱۳۴۰) و تصویرگ...,داستان کودک و نوجوانان
4,پرسش از غرب,«پرسش از غرب» به قلم دکتر اسماعیل شفیعی سروستا...,جامعه‌شناسی


In [3]:
from __future__ import unicode_literals
from hazm import *

normalizer = Normalizer()
stemmer = Stemmer()

In [4]:
normalizer.normalize(df['title'][0])

'ساختار نظریه\u200cهای جامعه\u200cشناسی ایران'

In [120]:
stemmer.stem('رفتم')

'رف'

In [121]:
word_tokenize(df['title'][1])

['جامعه', 'و', 'فرهنگ', 'کانادا']

In [122]:
df['categories'].value_counts()

رمان                      425
جامعه‌شناسی               425
کلیات اسلام               425
داستان کوتاه              425
داستان کودک و نوجوانان    425
مدیریت و کسب و کار        425
Name: categories, dtype: int64

In [5]:
ignore = ['»', '«', 'و', ',', 'را', 'از', '.', 'به',  '،', ':', '/']
ignore

['»', '«', 'و', ',', 'را', 'از', '.', 'به', '،', ':', '/']

In [6]:
novel = dict()
sociology = dict()
islamic = dict()
short_story = dict()
kids = dict()
business = dict()

In [7]:
novel.clear()
sociology.clear()
islamic.clear()
short_story.clear()
kids.clear()
business.clear()

for index, row in df.iterrows():
    tokens = word_tokenize(row['title'])
    #print(row['categories'])
    for token in tokens:
        word = token
        if (not word in ignore):
            #print(word)
            cat = row['categories']
            
            if (cat == 'رمان'):
                novel[word] = novel.get(word, 0) + 1
            if (cat == 'جامعه‌شناسی'):
                sociology[word] = sociology.get(word, 0) + 1
            if (cat == 'کلیات اسلام'):
                islamic[word] = islamic.get(word, 0) + 1
            if (cat == 'داستان کوتاه'):
                short_story[word] = short_story.get(word, 0) + 1
            if (cat == 'داستان کودک و نوجوانان'):
                kids[word] = kids.get(word, 0) + 1
            if (cat == 'مدیریت و کسب و کار'):
                business[word] = business.get(word, 0) + 1 



novel[x] is storing number of occurences of word x in a novel.

Since all the categories have same count, there is no difference in dividing this frequency number by their count.

In [8]:
tdf = pd.read_csv("books_test.csv")
tdf.head(5)

,title,description,categories
0,کآشوب,«کآشوب (بیست و سه روایت از روضه‌هایی که زندگی ...,داستان کوتاه
1,داستان‌های برق‌آسا,«داستان‌های برق‌آسا» نام مجموعه داستان‌هایی به...,داستان کوتاه
2,بحثی درباره مرجعیت و روحانیت,مجموعه مقالات «بحثی درباره مرجعیت و روحانیت» ش...,کلیات اسلام
3,قلعه‌ی حیوانات,«قلعه‌ی حیوانات» جورج اورول، درباره گروهی از ح...,رمان
4,قصه ما مثل شد (۱),«قصه ما مثل شد» یک مجموعه کتاب ۵ جلدی است که د...,داستان کودک و نوجوانان


In [10]:
from math import log

In [11]:
correct = 0

for index, row in tdf.iterrows():
    tokens = word_tokenize(row['title'])
    novel_prob = 0
    sociology_prob = 0
    islamic_prob = 0
    short_story_prob = 0
    kids_prob = 0
    business_prob = 0
    for token in tokens:
        word = token
        if (word in ignore):
            continue
            
        if (word in novel):
            novel_prob += log(novel[word])

        if (word in sociology):
            sociology_prob += log(sociology[word])

        if (word in islamic):
            islamic_prob += log(islamic[word])

        if (word in short_story):
            short_story_prob += log(short_story[word])

        if (word in kids):
            kids_prob += log(kids[word])

        if (word in business):
            business_prob += log(business[word])

    mx = max(novel_prob, sociology_prob, islamic_prob, short_story_prob, kids_prob, business_prob)
    if (row['categories'] == 'رمان' and mx == novel_prob):
        correct += 1

    if (row['categories'] == 'جامعه‌شناسی' and mx == sociology_prob):
        correct += 1

    if (row['categories'] == 'کلیات اسلام' and mx == islamic_prob):
        correct += 1

    if (row['categories'] ==  'داستان کوتاه' and mx == short_story_prob):
        correct += 1

    if (row['categories'] == 'داستان کودک و نوجوانان' and mx == kids_prob):
        correct += 1

    if (row['categories'] == 'مدیریت و کسب و کار' and mx == business_prob):
        correct += 1


In [12]:
print(correct)
print("Accuracy is ", correct / len(tdf))

336
Accuracy is  0.7466666666666667
